## Mockup of Upload mech

In [50]:
from ipywidgets import FileUpload, VBox, HTML, FloatText, Text, Layout, HBox, ToggleButton
from IPython.display import display, clear_output
from functools import partial

# Specify the allowed file extensions
allowed_extensions = ['.cif', '.xyz']

# Create FileUpload widget with the specified extensions
upload = FileUpload(accept=','.join(allowed_extensions), multiple=True)
upload_text = HTML()

# Dictionary to store uploaded files and associated float values
float_inputs = {}
file_names = {}
checkboxes = {}
file_states = {}

def create_file_widgets(file_info, file_widgets):
    file_name = file_info['name']

    # Add FloatText widget for ".cif" files
    is_cif = file_name.lower().endswith('.cif')
    checkboxes[file_name] = ToggleButton(
        value=True,
        description='',
        disabled=False,
        button_style='success',
        tooltip='',
        icon='check-square-o',
        layout=Layout(width='90px')
    )
    file_names[file_name] = Text(file_name, disabled=True, layout=Layout(width='500px'))
    float_inputs[file_name] = FloatText(value=5.0, disabled=not is_cif, 
                                        layout=Layout(width='150px', visibility = 'hidden' if not is_cif else 'visible'))
    
    file_widgets.append(HBox([checkboxes[file_name], file_names[file_name], float_inputs[file_name]],
                        layout=Layout(justify_content='flex-start')))
#     display(file_widgets)
    
    # Initialize state dictionary
    file_states[file_name] = True
    
    # Add an observer for each ToggleButton
    checkboxes[file_name].observe(partial(callback_toggle_button, file_name=file_name), names='value')

def callback_toggle_button(change, file_name):
    if change['owner'].value:
        change['owner'].button_style = 'success'
        change['owner'].icon = 'check-square-o'
    else:
        change['owner'].button_style = ''
        change['owner'].icon = 'minus'

def update_uploaded_files(change):
    # Display file widgets
    file_widgets = [HBox([
        Text("Include", layout=Layout(width='90px')),
        Text("Filename", layout=Layout(width='500px')),
        Text("Particle radius", layout=Layout(width='150px')),
    ], layout=Layout(justify_content='flex-start'))]
    
    num_files = len(upload.value)

    if num_files > 0:
        file_list = '<strong>Uploaded Files:</strong><br>'
        for file_info in upload.value:
            create_file_widgets(file_info, file_widgets)

    upload_text.value = file_list
    
    clear_output(wait=True)
    display(VBox([upload, upload_text], layout=Layout(justify_content='flex-start')))
    display(VBox(file_widgets))

# Observe changes in the FileUpload widget
upload.observe(update_uploaded_files, names='value')

# # Display the widgets
display(VBox([upload, upload_text], layout=Layout(justify_content='flex-start')))
# display(VBox(file_widgets))


### Handling of uploaded files (here .cif) using a temporary file system

In [51]:
from io import BytesIO
from ase.visualize import view
import tempfile
from debyecalculator.utility.generate import generate_nanoparticles

with tempfile.NamedTemporaryFile(delete=True, suffix='.cif', mode = "w") as temp_file:
    temp_filename = temp_file.name
    temp_file.write(BytesIO(upload.value[0]['content']).read().decode('utf-8'))
    temp_file.flush()

    out = generate_nanoparticles(temp_filename, radii=float_inputs[upload.value[0]['name']].value, _return_ase=True)
    
view(out[0].ase_structure, viewer='x3d')

### Mockup download button

In [52]:
import base64
import hashlib
import time
# from typing import Callable
import ipywidgets
from IPython.display import HTML, display

class DownloadButton(ipywidgets.Button):
    def __init__(self, filename: str, contents, **kwargs):
        super(DownloadButton, self).__init__(**kwargs)
        self.filename = filename
        self.contents = contents
        self.on_click(self.__on_click)

    def __on_click(self, b):
        contents = self.contents.encode('utf-8')
        b64 = base64.b64encode(contents)
        payload = b64.decode()
        digest = hashlib.md5(contents).hexdigest()  # bypass browser cache
        id = f'dl_{digest}'

        display(
            HTML(
                f"""
                    <html>
                    <body>
                    <a id="{id}" download="{self.filename}" href="data:text/csv;base64,{payload}" download>
                    </a>

                    <script>
                    (function download() {{
                    document.getElementById('{id}').click();
                    }})()
                    </script>

                    </body>
                    </html>
                """
            )
        )

In [49]:
DownloadButton(filename='data/output.txt', contents=str(out[0].ase_structure.positions), description='Download data')

DownloadButton(description='Download data', style=ButtonStyle())